In [1]:
import numpy as np
from stable_koopman_operator import StableKoopmanOperator
from quad import Quad
from task import Task, Adjoint

import matplotlib.pyplot as plt
import scipy.linalg
from group_theory import VecTose3, TransToRp, RpToTrans
from lqr import FiniteHorizonLQR
from quatmath import euler2mat 

Helper Functions
================

In [2]:
def get_measurement(x):
    g = x[0:16].reshape((4,4)) ## SE(3) matrix
    R,p = TransToRp(g)
    twist = x[16:]
    grot = np.dot(R, [0., 0., -9.81]) ## gravity vec rel to body frame
    return np.concatenate((grot, twist)) 

def get_position(x):
    g = x[0:16].reshape((4,4))
    R,p = TransToRp(g)
    return p

import numpy as np
import random

class ReplayBuffer:
    def __init__(self, capacity):
        self.capacity = capacity
        self.buffer = []
        self.position = 0

    def push(self, state, action, next_state):
        if len(self.buffer) < self.capacity:
            self.buffer.append(None)
        self.buffer[self.position] = (state, action, next_state)
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        batch = random.sample(self.buffer, batch_size)
        state, action, next_state = map(np.stack, zip(*batch))
        return state, action, next_state

    def __len__(self):
        return len(self.buffer)



In [3]:
quad = Quad() ### instantiate a quadcopter
replay_buffer = ReplayBuffer(100000)
### the timing parameters for the quad are used
### to construct the koopman operator and the adjoint dif-eq 
koopman_operator = StableKoopmanOperator(quad.time_step)
adjoint = Adjoint(quad.time_step)


In [4]:
np.abs(np.linalg.eig(koopman_operator.K)[0])

array([4.38003345, 4.38003345, 4.42549602, 4.42549602, 3.91210584,
       3.91210584, 5.06598428, 3.00773887, 3.00773887, 2.8390079 ,
       3.40143656, 3.40143656, 2.37738613, 2.37738613, 3.94200731,
       3.09744014, 2.79169213, 2.79169213, 1.15030063, 2.00807479,
       2.00807479, 0.33769495])

In [5]:
_R = euler2mat(np.random.uniform(-1.,1., size=(3,)))
_p = np.array([0., 0., 0.])
_g = RpToTrans(_R, _p).ravel()
_twist = np.random.uniform(-1., 1., size=(6,)) #* 2.0
state = np.r_[_g, _twist]

In [6]:
batch_size=64

for t in range(1000):
    m_state = get_measurement(state)
    ustar = np.random.normal(0., 0.2, size=(4,))
    next_state = quad.step(state, ustar)
    
    replay_buffer.push(m_state, ustar, get_measurement(next_state))
    if len(replay_buffer) > batch_size:
        input_data, control_data, output_data = replay_buffer.sample(batch_size)
        koopman_operator.compute_operator_from_data(input_data, control_data, 
                                                        output_data)
    state = next_state

32.49268559654457
9.964692770929267
7.901416874658519
6.188204967221781
5.166875921214798
4.894956862634698
4.449778287878664
3.9795751557897563
3.577769230077607
3.4627205602562685
3.4627205602562685
31.13496423512819
10.305834197712148
8.1239490709596
6.47203167961267
5.502742901604829
5.143204088184429
4.661873772126153
4.55609232718105
4.528502523651477
4.136251733081535
4.136251733081535
25.596415533851424
9.602139917652956
6.677609056379115
5.535845690771257
4.828188216777593
4.548359603761818
3.9437288650980658
3.8588903653682705
3.5920352004218574
3.0343331589839004
3.0343331589839004
27.518024102530273
11.989844405078102
9.189283578648125
7.91585028961077
6.87919037771743
6.452137084660097
5.727776837577994
5.494427308528267
5.213949584429443
4.414436049919441
4.414436049919441
31.258358459712323
9.17171158727909
7.779471194196696
6.010218375760902
4.939898878809266
4.775275298072827
4.189374252573979
4.091214380120641
3.2680240297062846
2.8400087676018297
2.8400087676018297
2

KeyboardInterrupt: 

In [9]:
output_data.shape

(32, 4)

In [7]:
np.abs(np.linalg.eig(koopman_operator.K)[0])

array([2.63462303e-17, 9.99983183e-01, 6.74816689e-17, 3.84552189e-17,
       1.50573735e-17, 9.70101371e-18, 5.21248054e-18, 4.75279010e-18,
       2.39878693e-18, 2.23012645e-18, 2.16871328e-18, 2.12969892e-18,
       1.71624728e-18, 1.28426544e-18, 1.08520593e-18, 5.06608973e-19,
       4.55842193e-19, 5.76707533e-19, 3.50158960e-19, 1.87522905e-19,
       8.15118789e-20, 1.70662771e-19])

In [8]:
np.linalg.eig(koopman_operator.Kx)[0]

array([-3.78174352e+00  +0.j        , -9.13194503e+03  +0.j        ,
       -8.66731243e+03  +0.j        , -8.49670322e+03+300.01507283j,
       -8.49670322e+03-300.01507283j, -8.44649539e+03  +0.j        ,
       -8.20784850e+03+304.39593915j, -8.20784850e+03-304.39593915j,
       -8.03443809e+03+315.45823487j, -8.03443809e+03-315.45823487j,
       -7.45223206e+03  +0.j        , -7.55491808e+03 +73.3284287j ,
       -7.55491808e+03 -73.3284287j , -7.62669094e+03  +0.j        ,
       -7.83535549e+03 +36.89041868j, -7.83535549e+03 -36.89041868j,
       -7.83517113e+03  +0.j        , -8.12173620e+03  +0.j        ])